# 04_Transfer_learning_in_TF_part_1_feature_extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.



In [ ]:
!nvidia-smi

## Dowloading and becoming one with the data

In [ ]:
# Get data (10% of 10 food classes from Food101)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f'There are {len(dirnames)} directories and {len(filenames)} images in {dirpath} .')

##  Preparing the data (Creating data loaders)

We'll use the `ImageDataGenerator` to load in our images in batches.

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test/'

train_datagen = ImageDataGenerator(rescale= 1/255.)
test_datagen = ImageDataGenerator(rescale= 1/255.)

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                                                              target_size= IMAGE_SHAPE,
                                                                                              batch_size= BATCH_SIZE,
                                                                                              class_mode='categorical')

print('Testing images:')
test_data = test_datagen.flow_from_directory(train_dir,
                                             target_size= IMAGE_SHAPE,
                                             batch_size= BATCH_SIZE,
                                             class_mode='categorical')


## Setting up callbacks

In [ ]:
# Create TensorBoard callback (We'll usea a fuction because we need to reate a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback


## Creating models using TensorFlowHub.

we're going to use two models from TensorFlow Hub:

* ResNetV2 - a state of the art computer vision model architecture from 2016.
* EfficientNet - a state of the art computer vision architecture from 2019.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [ ]:
# Let's make a create_model () function to create a model from a URL
def create_model(model_url, num_classes=10):
  '''
  Creates a keras Sequential with the TensorFlow Hub Url
  
  Args:
   model_url (str) : A TensorFlow Hub feature extraction URL.
   num_classes (int): Number of output neurons in the ouput layer.
   
  Returns:
    An uncompiled Keras Sequential model with model_url as feature extactor layer and Dense output layer.
  '''
  # Download the pretrained model
  feature_extractor_layer = hub.KerasLayer(model_url,
                          trainable = False,
                          name= 'feature_extraction_layer',
                         input_shape = IMAGE_SHAPE+(3,))
  
  # Create or own model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation= 'softmax', name= 'output_layer')
    ])

  return model

                    

### Creating and testing  Resnet TensorFlow Hub Feature Extraction

In [ ]:
resnet_model = create_model(resnet_url,
                                            num_classes = train_data_10_percent.num_classes)

In [ ]:
train_data_10_percent.num_classes

In [ ]:
resnet_model.summary()

In [ ]:
# Compile our resnet model
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer= tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
# Let's fit our Resnet model to the data (10 percent of 10 classes)
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data= test_data,
                                  validation_steps= len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name= 'tensorflow_hub',
                                                                         experiment_name="resnet50V2"
                                                                         )])

In [ ]:
# Let's create a function to plot our loss curves.
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(resnet_history)

### Creating and testing EfficientNetB0 TensorFlow Hub Feature Extraction model.

In [ ]:
# Create EfficinetNetB0 feature extractor model
efficientnet_model = create_model(model_url= efficientnet_url,
                                  num_classes= train_data_10_percent.num_classes)

# Compile EfficienteNet model
efficientnet_model.compile(loss= "categorical_crossentropy",
                           optimizer = tf.keras.optimizers.Adam(),
                           metrics= ["accuracy"]) 

# Fit EfficientNet model to 10% of training data
efficientnet_history = efficientnet_model.fit(train_data_10_percent,
                                              epochs=5,
                                              steps_per_epoch= len(train_data_10_percent),
                                              validation_data = test_data,
                                              validation_steps= len(test_data),
                                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                                     experiment_name = "efficientnetb0")])
                                      

In [ ]:
efficientnet_model.summary()

# Comparing the two models in tensorboard

In [ ]:
# Upload tensorBoard dev records.
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EfficientNetB0 vs. ResNet50V2" \
  --description "Comparing two different TF Hub feature extraction models architectures using 10% of training images" \
  --one_shot